# 🦌 ELK Transformer 🤖
A transformer object that will convert some input source into valid Elk Json.
This example using a transformer around a networkx graph.

In [ ]:
import networkx, elk, elk.nx, elk.tools, ipywidgets, pathlib, json

## Flat structure

A `networkx.MultiDiagraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect edges between the nodes_

In [ ]:
flat_graph = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("flat_graph.json").read_text())
)

flat_xelk = elk.nx.XELK(source=(flat_graph, None))
flat_elk = elk.ElkDiagram(flat_xelk.to_dict())
flat_elk

## Hierarchical Diagram with Ports 

In [ ]:
hier_tree = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_tree.json").read_text())
)
hier_ports = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("hier_ports.json").read_text())
)

hier_xelk = elk.nx.XELK(source=(hier_ports, hier_tree))
hier_elk = elk.ElkDiagram(hier_xelk.to_dict())

toggle = ipywidgets.Button(description="Toggle Collapsed")

@toggle.on_click
def toggle_node(widget):
    for element_id in hier_elk.selected:
        if element_id in hier_tree:
            for child in hier_tree.neighbors(element_id):
                state = hier_tree.nodes[child].get('hidden', False)
                hier_tree.nodes[child]['hidden'] = not state
            hier_elk.value = hier_xelk.to_dict()

ipywidgets.VBox([
    ipywidgets.HBox([
        ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), 
        toggle
    ]), 
    hier_elk
])

## 🦌 Learn More 📖 

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)